In [26]:
import os
import json

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from torch.utils import data
device = torch.device('cuda')

# nombre max de frame à utiliser lors des convolutions 3d
max_frame = 60

https://stanford.edu/~shervine/blog/pytorch-how-to-generate-data-parallel

https://pytorch.org/docs/stable/data.html

https://github.com/kenshohara/3D-ResNets-PyTorch

In [27]:
list_IDs = {}
with open("data/dataset.json", "r") as file:
    list_IDs = json.load(file)
labels = {}
with open("data/labels_10cl.json", "r") as file:
    labels = json.load(file)

In [32]:
dataloader_config = {
    "batch_size": 32,
    "shuffle": True,
    "num_workers": 4,
    "pin_memory": False,
    "drop_last": True
}

In [33]:
class NTUDataset(data.Dataset):
    def __init__(self, list_IDs, labels, path, max_frame=max_frame):
            self.max_frame = max_frame
            self.path = path
            self.labels = labels
            self.list_IDs = list_IDs

    def __len__(self):
        return len(self.list_IDs)

    def __getitem__(self, index):
        # Select sample
        ID = self.list_IDs[index]
        array = np.load(self.path + ID + '.npy')
        if not(self.max_frame is None):
            mid_frame = array.shape[0] // 2
            array = array[mid_frame-self.max_frame//2:mid_frame+self.max_frame//2]
        # Load data and get label
        X = torch.from_numpy(array)
        y = self.labels[ID]

        return X, y
    
trainset = NTUDataset(list_IDs["train"], labels, path="data/processed/train/")
testset = NTUDataset(list_IDs["validation"], labels, path="data/processed/test/")

train_gen = data.DataLoader(trainset, **dataloader_config)
test_gen = data.DataLoader(testset, **dataloader_config)

In [34]:
class SpatialClassifier(nn.Module):
    
    def __init__(self):
        super(SpatialClassifier, self).__init__()
        self.conv11 = nn.Conv3d(max_frame, 64, 3)
        self.conv12 = nn.Conv3d(64, 64, 3)
        
        self.conv21 = nn.Conv3d(128, 128, 3)
        self.conv22 = nn.Conv3d(128, 128, 3)

        self.conv31 = nn.Conv3d(256, 256, 3)
        self.conv32 = nn.Conv3d(256, 256, 3)
        
        self.fc = nn.Linear(256*6*6, 11)
        
    def forward(self, x):
        x = F.relu(self.conv11(x))
        x = F.relu(self.conv22(x))
        x = F.max_pool3d(2)
        
        x = F.relu(self.conv21(x))
        x = F.relu(self.conv22(x))
        x = F.max_pool3d(2)
        
        x = F.relu(self.conv31(x))
        x = F.relu(self.conv32(x))
        x = F.max_pool3d(2)
        
        x = x.view(-1, self.num_flat_features(x))
        x = self.fc(x)
        return x
    
    def num_flat_features(self, x):
        size = x.size()[1:]  # all dimensions except the batch dimension
        num_features = 1
        for s in size:
            num_features *= s
        return num_features

In [35]:
net = SpatialClassifier()
net.to(device)
print(net)

AssertionError: Torch not compiled with CUDA enabled

In [22]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

In [23]:
running_loss = 0.0
for i, data in enumerate(train_gen, 0):
    # get the inputs; data is a list of [inputs, labels]
    inputs, labels = data

    # zero the parameter gradients
    optimizer.zero_grad()

    # forward + backward + optimize
    outputs = net(inputs)
    loss = criterion(outputs, labels)
    loss.backward()
    optimizer.step()

    # print statistics
    running_loss += loss.item()
    if i % 2000 == 1999:    # print every 2000 mini-batches
        print('[%d, %5d] loss: %.3f' %
              (epoch + 1, i + 1, running_loss / 2000))
        running_loss = 0.0

print('Finished Training')

BrokenPipeError: [Errno 32] Broken pipe